In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
def SIR(s0,i0,r0, pC, gamma, t_max, stepsize):
    T = np.arange(0,t_max+stepsize,stepsize)
    G = len(s0)
    s = np.zeros([len(T),G])
    i = np.zeros([len(T),G])
    r = np.zeros([len(T),G])

    for idx,t in enumerate(T):
        if idx==0:
            s[idx] = s0
            i[idx] = i0
            r[idx] = r0
        else:
            Q = np.diag(s[idx-1,:]) @ pC @ i[idx-1,:]
            ds_dt = -Q
            di_dt = Q - gamma*i[idx-1,:]
            dr_dt = gamma*i[idx-1,:]
            
            s[idx] = s[idx-1,:] + ds_dt * stepsize
            i[idx] = i[idx-1,:] + di_dt * stepsize
            r[idx] = r[idx-1,:] + dr_dt * stepsize
    
    return s, i, r, T

In [ ]:
# SIR(S0,I0,R0, pC, gamma, t_max, stepsize):
pC = [[0.45, 0.45, 0.45, 0.45],
     [0.9, 0.9, 0.9, 0.9],
     [1.35, 1.35, 1.35, 1.35],
     [1.8, 1.8, 1.8, 1.8]]
S,I,R,T = SIR([0.999,0.999, 0.999, 0.999],[0.001,0.001,0.001,0.001],[0,0,0,0],
             pC,3,20,0.05)
fig, ax = plt.subplots()
alpha = [0.3, 0.5, 0.7, 1.0]
for group in [0,1, 2, 3]:
  ax.plot(T,I[:,group], color='blue', alpha = alpha[group], label=f'Group {group + 1}')
ax.set_xlabel('time')
ax.set_ylabel('proportion of people')
ax.legend()
ax.set_ylim(bottom=0)
ax.set_xlim(left=0)
ax.set_title("i compartments for each group over time")

In [ ]:
def avgSus(p, s, T):
  # p should be an array of susceptibilities for each group
  # T is an array of times
  allresult=[]
  
  for t in range(0, len(T)):
    num = 0
    den = 0
    for i in range(0, 4):
      num += p[i] * s[t, i]
      den += s[t, i]
      
    allresult.append(num/den)
  
  return allresult
  
fig, ax = plt.subplots()
alpha = [0.3, 0.5, 0.7, 1.0]
p = [1, 2, 3, 4]
for group in [0,1, 2, 3]:
  ax.plot(T,S[:,group], color='green', alpha = alpha[group], label=f'Group {group + 1}')
ax.plot(T,avgSus(p, S, T), color='black', label='Average relative susceptibility')
ax.set_xlabel('time')
ax.set_ylabel('proportion of people')
ax.legend()
ax.set_ylim(bottom=0)
ax.set_xlim(left=0)
ax.set_title("s compartments for each group over time")

In [ ]:
from scipy.stats import nbinom
import math

def branching(G, R0, k):
  mean = R0
  variance = mean + (mean**2)/k
  p = mean/variance
  n = mean**2 / (variance - mean)

  extinctions = 0
  
  for i in range(1000):
    total_inf = 1
    
    for g in range(G):
      if total_inf == 0:
        break
        
      secondary_inf = nbinom.rvs(n=n * total_inf, p=p)
      total_inf = secondary_inf
      
      if total_inf > 100000000:
        break
    
    if total_inf == 0:
      extinctions += 1
      
  return extinctions / 1000

k = [0.1, 0.5, 1.0, 5.0, 10.0] # dispersion parameter k
R0 = 3 # mean R0
G = 100
for kval in k:
  q = branching(G, R0, kval)
  print(kval, ": ", q)